## Penguin Classifier

Chapter 2 of the fastai book details and Image Classifier with a bear example. We are prompted to create our own classifier model try out deployment with an app.

The aim of this project is to create a model to classify three penguin species. Using Bing Search to download images as described in the book, a CNN learner is fine-tuned, and the project deployed using an in-notebook application.

In [1]:
from fastbook import *
from fastai.vision.widgets import *

To set up the Bing Search, sign into Microsoft Azure account and paste your key in as the second argument. We then specify the species of penguins in `penguin_types` and create a folder called Penguins to store the downloaded images.

In [2]:
key = os.environ.get('AZURE_SEARCH_KEY', 'f545086c6fb74ceeb77ac5f3bbe0d81d')
penguin_types = 'african','king','emporer'
path = Path('Penguins')

Create a for loop that searches Bing for each penguin species specified in `penguin_types`, and downloads these images into seperate folders within the Penguin folder.

In [3]:
if not path.exists():
    path.mkdir()
    for o in penguin_types:
        dest = (path/o) # initiate naming for the folders for each species
        dest.mkdir(exist_ok=True) # create folders
        results = search_images_bing(key, f'{o} penguin') # search Bing for images
        download_images(dest, urls=results.attrgot('contentUrl')) # download images to the destination folder

Our folder has image files in it, we want to make sure that all images used aren't corrupt. `verify_images` checks this for us, and to remove them, we `unlink` each one from the folder.

In [4]:
fns = get_image_files(path)
failed = verify_images(fns)
failed.map(Path.unlink);

Now, we will create a `DataBlock` object. This is like a template for creating a `Dataloaders` that will tell fastai four things about the data:
- What kind of data we have
- How to get the items
- How to split the data into training and validation sets
- How to label the data

In [7]:
penguins = DataBlock(
    blocks=(ImageBlock, CategoryBlock), # independent variable and dependent variable (african, king, emporer)
    get_items=get_image_files, # how to get the files
    splitter=RandomSplitter(valid_pct=0.2, seed=42), # how to split the data - split data randomly, with 20% validation
    get_y=parent_label, # how to label the data - parent_label gets the name of the folder it is in
    item_tfms=Resize(128)) # function applied to each image - resize them to 128x128 pixels

As our dataset is quite small - 150 images of each species at most - we'll use `RandomResizedCrop` on each image size of 224 pixels (which is standard for image classification) and the default `aug_transforms` on the batch. These batch augmentations include image rotation, warping and contrast changes.

In [9]:
penguins = penguins.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = penguins.dataloaders(path)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


Now to train the classifier. We can now create the `Learner` as a convolutional neural network and specify the architecture.

The learner function tries to figure out what the parameters are that best cause the `Dataloaders` to match the labels in the dataset. `resnet` refers to the number of layers in the architecture. The more layers, the longer it'll take, and the more prone to overfitting. Thus, we've gone with `resnet18` - out of the options 18, 34, 50, 101, 152. Then, we define the metric, or the measure of the quality of the model's prediction using the validation set, as `error_rate`. This function will provide the percentage of images in the validation set that are being classified incorrectly after every epoch.

In order to fit the model, you need to define how many times each image should be looked at - the number of epochs. This number depends on how much time you have, starting small is okay as you can train it with more later if you have the time. `fine_tune` is used when we are transfer learning - using a pretrained model for a task that is different to what it was originally trained for.

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

Now that the model has been trained, we can visualise the count of correctly and incorrectly classified images with a confusion matrix. Ideally, we want the negative diagonal to be darkest and add to the total number of observations in our validation set, while the surrounding boxes are light and equal to zero. These off-diagonal boxes represent the model's errors.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

Loss is a penalty representing how bad the model's incorrect prediction was. We can use `plot_top_losses` to display the images with the highest loss in our dataset to help us distinguish whether the errors are due to a model problem or dataset problem. The values above each image represent the class the model predicted, the actual class, the loss value and the model's probability, or confidence level.

In [ ]:
interp.plot_top_losses(5, nrows=1)

The chapter explains that intuitively, data cleaning is conducted before the model is trained. However, in the example, their model has a wrongly classified prediction with high confidence, and it is found that the image was incorrectly labelled to begin with. This reinforces the idea that sometimes, a model can help you find data issues more easily. Thus, they prefer to train a simple model first to assist with data cleaning.

In the case where we may need to change the label of an image, `ImageClassifierCleaner` is a GUI (graphical user interface) to manually relabel or remove images from both the training and validation sets.

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

We can use the `export` method to save our `Learner` as a file to access later called "export.pkl". This will save the model architecture, parameters and the `Dataloaders` so you don't have to define these again.

In [ ]:
learn.export()

For deployment, we can create a basic GUI application in the notebook. This will use the model to give a prediction for an image that the user uploads. We will initialise all the bits and pieces we need for it:
- A file upload button
- An output widget to display the uploaded image
- A run button that will initiate the prediction
- A label to display the model's prediction of the image

In [ ]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
btn_run = widgets.Button(description='Classify')
lbl_pred = widgets.Label()

We'll create a function called a click event handler is called whenever the button is clicked.

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1]) # create an image from the upload
    out_pl.clear_output() # clear the output
    with out_pl: display(img.to_thumb(128,128)) # display the image
    pred,pred_idx,probs = learn_inf.predict(img) # call the prediction
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}' # display the prediction

btn_run.on_click(on_click_classify) # on click, run the function

Finally, we'll pull all these together into a vertical box, `VBox`. And now our GUI is complete!

In [ ]:
VBox([widgets.Label('Select your penguin!'), 
      btn_upload, btn_run, out_pl, lbl_pred])